<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/TaylorTest3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Google Drive 연결 및 필요한 라이브러리 불러오기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder

In [4]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_motor10'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            data = pd.read_csv(file_path, header=None).values
            data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
            data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
            X.append(data)
            y.append(category)
    return np.array(X), np.array(y)

# 데이터 로드
X_train, y_train = load_data(base_dir, 'train')
X_val, y_val = load_data(base_dir, 'validation')
X_test, y_test = load_data(base_dir, 'test')

# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded)
y_val_categorical = tf.keras.utils.to_categorical(y_val_encoded)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded)

In [5]:
# 모델 정의
model = models.Sequential()

# Conv1 레이어
model.add(layers.Conv1D(filters=64, kernel_size=16, strides=16, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
# Pool1 레이어
model.add(layers.MaxPooling1D(pool_size=2, strides=2))

# Conv2 레이어
model.add(layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu'))

# Conv3 레이어
model.add(layers.Conv1D(filters=64, kernel_size=5, strides=1, activation='relu'))

# Conv4 레이어
model.add(layers.Conv1D(filters=128, kernel_size=5, strides=1, activation='relu'))

# Pool2 레이어
model.add(layers.MaxPooling1D(pool_size=2, strides=2))

# Flatten 레이어
model.add(layers.Flatten())

# FC1 레이어
model.add(layers.Dense(units=5000, activation='relu'))

# FC2 레이어
model.add(layers.Dense(units=1000, activation='relu'))

# Output 레이어
model.add(layers.Dense(len(categories), activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 출력
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 750, 64)           2112      
                                                                 
 max_pooling1d (MaxPooling1  (None, 375, 64)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 373, 32)           6176      
                                                                 
 conv1d_2 (Conv1D)           (None, 369, 64)           10304     
                                                                 
 conv1d_3 (Conv1D)           (None, 365, 128)          41088     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 182, 128)          0         
 g1D)                                                   

In [6]:
# 모델 훈련
history = model.fit(X_train, y_train_categorical, epochs=10, validation_data=(X_val, y_val_categorical))

Epoch 1/10
394/394 [==============================] - 203s 513ms/step - loss: 0.9788 - accuracy: 0.6182 - val_loss: 0.1491 - val_accuracy: 0.9467
Epoch 2/10
394/394 [==============================] - 199s 505ms/step - loss: 0.0217 - accuracy: 0.9935 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 3/10
394/394 [==============================] - 202s 512ms/step - loss: 0.0592 - accuracy: 0.9859 - val_loss: 0.4592 - val_accuracy: 0.8133
Epoch 4/10
394/394 [==============================] - 204s 517ms/step - loss: 0.1577 - accuracy: 0.9604 - val_loss: 0.0251 - val_accuracy: 0.9956
Epoch 5/10
394/394 [==============================] - 201s 511ms/step - loss: 0.0072 - accuracy: 0.9980 - val_loss: 2.6239e-04 - val_accuracy: 1.0000
Epoch 6/10
394/394 [==============================] - 202s 512ms/step - loss: 0.0046 - accuracy: 0.9989 - val_loss: 2.2455e-04 - val_accuracy: 1.0000
Epoch 7/10
394/394 [==============================] - 201s 509ms/step - loss: 0.0125 - accuracy: 0.9971 - val_loss: 

In [7]:
# 모델 평가 (훈련 후)
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy before pruning: {test_accuracy}')

85/85 [==============================] - 3s 38ms/step - loss: 0.1222 - accuracy: 0.9478
Test accuracy before pruning: 0.9477777481079102


In [8]:
# 원본 모델 저장
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
# 모델 사이즈 측정
model_size = os.path.getsize('model.h5') / (1024 * 1024)
print(f"Model Size before pruning: {model_size:.2f} MB")

Model Size before pruning: 1391.09 MB


In [10]:
# 프루닝 전 전체 가중치 수와 0이 아닌 가중치 수를 계산
total_weights = 0
non_zero_weights = 0
for weight in model.trainable_variables:
    w = weight.numpy()
    total_weights += w.size
    non_zero_weights += np.count_nonzero(w)
print(f"Before pruning: Non-zero weights = {non_zero_weights}, Total weights = {total_weights}")

Before pruning: Non-zero weights = 121549677, Total weights = 121549684


In [11]:
# === 테일러 전개 기반 프루닝 구현 ===
def taylor_pruning(model, X_sample, y_sample, pruning_threshold):
    # 손실에 대한 가중치의 변화도 계산
    with tf.GradientTape() as tape:
        predictions = model(X_sample)
        loss = tf.keras.losses.categorical_crossentropy(y_sample, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    # 중요도 계산: |w_i * grad_w_i|
    importance_scores = []
    weight_tensors = []
    grad_tensors = []
    for weight, grad in zip(model.trainable_variables, gradients):
        # 편향(bias) 제외 (필요한 경우)
        if 'bias' in weight.name:
            continue
        if weight is not None and grad is not None:
            importance = tf.abs(weight * grad)
            importance_scores.extend(importance.numpy().flatten())
            weight_tensors.append(weight)
            grad_tensors.append(grad)

    # 중요도 점수를 NumPy 배열로 변환
    all_scores = np.array(importance_scores)

    # 중요도 점수 통계 출력
    print(f"Importance Scores - min: {all_scores.min()}, max: {all_scores.max()}, mean: {all_scores.mean()}")

    # 프루닝 임계값 출력
    print(f"Pruning Threshold (manually set): {pruning_threshold}")

    # 임계값 이하의 가중치 프루닝
    for weight, grad in zip(weight_tensors, grad_tensors):
        importance = tf.abs(weight * grad)
        mask = tf.cast(importance >= pruning_threshold, weight.dtype)
        pruned_weight = weight * mask
        weight.assign(pruned_weight)

# 변화도 계산을 위한 샘플 배치 사용
X_sample = X_train
y_sample = y_train_categorical

In [12]:
# 프루닝 임계값 직접 설정
pruning_threshold = 0.00001  # 직접 임계값 설정

taylor_pruning(model, X_sample, y_sample, pruning_threshold=pruning_threshold)

Importance Scores - min: 0.0, max: 569.6116333007812, mean: 0.0010790671221911907
Pruning Threshold (manually set): 1e-05


In [13]:
# 프루닝 후 가중치 상태 확인
total_weights_after = 0
non_zero_weights_after = 0
for weight in model.trainable_variables:
    w = weight.numpy()
    total_weights_after += w.size
    non_zero_weights_after += np.count_nonzero(w)
print(f"After pruning: Non-zero weights = {non_zero_weights_after}, Total weights = {total_weights_after}")
print(f"Final non-zero weight ratio: {non_zero_weights_after / total_weights_after:.4f}")

After pruning: Non-zero weights = 2165683, Total weights = 121549684
Final non-zero weight ratio: 0.0178


In [14]:
# 각 레이어별로 0의 비율 출력
for layer in model.layers:
    weights = layer.get_weights()
    if len(weights) > 0:
        weight = weights[0]
        zero_count = np.sum(weight == 0)
        total_count = weight.size
        zero_ratio = (zero_count / total_count) * 100
        print(f"Layer {layer.name}: {zero_ratio:.2f}% of weights are zero.")

Layer conv1d: 45.90% of weights are zero.
Layer conv1d_1: 73.19% of weights are zero.
Layer conv1d_2: 77.24% of weights are zero.
Layer conv1d_3: 89.07% of weights are zero.
Layer dense: 98.38% of weights are zero.
Layer dense_1: 94.78% of weights are zero.
Layer dense_2: 69.97% of weights are zero.


In [15]:
# 프루닝된 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy after pruning: {test_accuracy}')

85/85 [==============================] - 3s 38ms/step - loss: 0.1227 - accuracy: 0.9478
Test accuracy after pruning: 0.9477777481079102


In [16]:
# 모델 재컴파일 (필요한 경우)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# 미세조정 훈련
fine_tune_epochs = 10
history_finetune = model.fit(X_train, y_train_categorical, epochs=fine_tune_epochs, validation_data=(X_val, y_val_categorical))

Epoch 1/10
394/394 [==============================] - 206s 518ms/step - loss: 0.0551 - accuracy: 0.9875 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 2/10
394/394 [==============================] - 201s 509ms/step - loss: 0.0039 - accuracy: 0.9992 - val_loss: 1.8683e-04 - val_accuracy: 1.0000
Epoch 3/10
394/394 [==============================] - 202s 512ms/step - loss: 5.8661e-05 - accuracy: 1.0000 - val_loss: 7.8689e-05 - val_accuracy: 1.0000
Epoch 4/10
394/394 [==============================] - 203s 515ms/step - loss: 3.9509e-05 - accuracy: 1.0000 - val_loss: 7.4252e-05 - val_accuracy: 1.0000
Epoch 5/10
394/394 [==============================] - 202s 512ms/step - loss: 0.2224 - accuracy: 0.9787 - val_loss: 0.0105 - val_accuracy: 0.9989
Epoch 6/10
394/394 [==============================] - 204s 518ms/step - loss: 7.6760e-04 - accuracy: 0.9999 - val_loss: 6.9323e-05 - val_accuracy: 1.0000
Epoch 7/10
394/394 [==============================] - 206s 522ms/step - loss: 4.5478e-05 - accur

In [18]:
# 재학습 후 모델 평가
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy after fine-tuning: {test_accuracy}')

85/85 [==============================] - 3s 38ms/step - loss: 1.1045e-05 - accuracy: 1.0000
Test accuracy after fine-tuning: 1.0


In [19]:
# 재학습된 모델 저장
model.save('model_pruned_finetuned.h5')

In [20]:
# 재학습된 모델 사이즈 측정
model_size = os.path.getsize('model_pruned_finetuned.h5') / (1024 * 1024)
print(f"Model Size after fine-tuning: {model_size:.2f} MB")

Model Size after fine-tuning: 1391.09 MB


In [21]:
# 프루닝 후 가중치 상태 확인
total_weights_after = 0
non_zero_weights_after = 0
for weight in model.trainable_variables:
    w = weight.numpy()
    total_weights_after += w.size
    non_zero_weights_after += np.count_nonzero(w)
print(f"After pruning: Non-zero weights = {non_zero_weights_after}, Total weights = {total_weights_after}")
print(f"Final non-zero weight ratio: {non_zero_weights_after / total_weights_after:.4f}")

After pruning: Non-zero weights = 8756519, Total weights = 121549684
Final non-zero weight ratio: 0.0720
